In [1]:
import numpy as np
import matplotlib.pyplot as plt 
plt.rcParams['figure.facecolor'] = "white"

from reservoir import reservoir as res
from data import data
from error import error

from reservoir.reservoir import modify_node

Create some test data

In [2]:
DEL_T = 0.01

In [3]:
lorenz_data = data.generate_lorenz_63(params=[10, 28, -8/3], 
                                      initial_state=[1, 1, 1], del_t=DEL_T,
                                      steps=30000, washout=10000)
print(lorenz_data)
print(lorenz_data.shape)
training_data = lorenz_data[:lorenz_data.shape[0] // 2]
test_data = lorenz_data[lorenz_data.shape[0] // 2:]

[[ -8.47965722 -14.5123554   15.72421705]
 [ -9.09696196 -15.40313805  16.60822317]
 [ -9.73928527 -16.26936278  17.64384007]
 ...
 [ -9.63164986  -0.84252354  36.6800054 ]
 [ -8.75710572  -0.08145599  35.7558209 ]
 [ -7.90106831   0.52294291  34.79606715]]
(20000, 3)


In [4]:
hyperparams = {
    'GAMMA': 7.7,
    'SIGMA': 0.81,
    'RHO_IN': 0.37,
    'K': 3,
    'RHO_R': 0.41
}

# number of reservoir nodes
N = 100
# dimension of input signal
d = 3

Internal reservoir construction.

In [5]:
W_r = res.generate_W_r(hyperparams, (N, N), seed=42)
W_in = res.generate_W_in(hyperparams, (N, d), seed=42)

Generate reservoir.

In [6]:
def next_training_node(r_prev: np.ndarray, u_prev: np.ndarray, 
                       hyperparams: dict, W_r: np.ndarray, 
                       W_in: np.ndarray, delta_t: np.double) -> np.ndarray:
# parse hyperparameter
    GAMMA = hyperparams["GAMMA"]
    
    return r_prev + delta_t * (((-GAMMA) * r_prev) + GAMMA * np.tanh(np.dot(W_r, r_prev) + np.dot(W_in, u_prev)))

In [7]:
# using import
import_modified_train_res, import_train_res = res.generate_training_reservoir(
    data=training_data,
    hyperparams=hyperparams,
    W_r=W_r,
    W_in=W_in,
    delta_t=DEL_T
)

[ 1.25784353 -1.14952156  0.2232467   3.39911833 -0.4099762   0.58150832
  5.47354794  0.20400196 -0.61734167 -0.86785125  1.70013248 -1.96109681
  0.5523374   1.97627089  4.54923585  6.13125458 -4.05184532 -0.14155296
 -0.9233338   2.10341081  1.6034256  -0.061099   -2.16087584  2.47035905
  0.06896331  4.56136794 -5.45963017 -2.58172188  3.01185164  2.09271969
  3.59110479 -2.66271615  1.53287196  0.3279889   3.7957388  -0.42170341
 -2.60743298  0.82388388 -5.43361392 -0.88775517  1.59369119 -0.81030227
 -0.56294716  6.20710117  0.20025215 -3.59746505  0.259477    1.39149207
  1.61993173 -3.11775503  1.11109864 -2.58381244 -2.24740834 -1.78756819
  1.60726667  9.31186998 -4.29413815  0.53331956 -5.81558309  0.17620202
 -2.58166629  0.93275271  0.08439438  0.96887137 -0.47639255 -0.51977737
 -1.41891925 -0.65646523  4.62694312  0.59568138  0.08322867 -0.71545454
  2.64035851 -3.0297703   5.74449294  1.28967993  2.04112466 -0.13838184
 -0.11839749 -0.9750331  -2.15042054  5.37309278  2

In [8]:
print(np.dot(W_in, training_data[0]))

[ 1.25784353 -1.14952156  0.2232467   3.39911833 -0.4099762   0.58150832
  5.47354794  0.20400196 -0.61734167 -0.86785125  1.70013248 -1.96109681
  0.5523374   1.97627089  4.54923585  6.13125458 -4.05184532 -0.14155296
 -0.9233338   2.10341081  1.6034256  -0.061099   -2.16087584  2.47035905
  0.06896331  4.56136794 -5.45963017 -2.58172188  3.01185164  2.09271969
  3.59110479 -2.66271615  1.53287196  0.3279889   3.7957388  -0.42170341
 -2.60743298  0.82388388 -5.43361392 -0.88775517  1.59369119 -0.81030227
 -0.56294716  6.20710117  0.20025215 -3.59746505  0.259477    1.39149207
  1.61993173 -3.11775503  1.11109864 -2.58381244 -2.24740834 -1.78756819
  1.60726667  9.31186998 -4.29413815  0.53331956 -5.81558309  0.17620202
 -2.58166629  0.93275271  0.08439438  0.96887137 -0.47639255 -0.51977737
 -1.41891925 -0.65646523  4.62694312  0.59568138  0.08322867 -0.71545454
  2.64035851 -3.0297703   5.74449294  1.28967993  2.04112466 -0.13838184
 -0.11839749 -0.9750331  -2.15042054  5.37309278  2

In [9]:
import_train_res[0]

array([ 1.25784353, -1.14952156,  0.2232467 ,  3.39911833, -0.4099762 ,
        0.58150832,  5.47354794,  0.20400196, -0.61734167, -0.86785125,
        1.70013248, -1.96109681,  0.5523374 ,  1.97627089,  4.54923585,
        6.13125458, -4.05184532, -0.14155296, -0.9233338 ,  2.10341081,
        1.6034256 , -0.061099  , -2.16087584,  2.47035905,  0.06896331,
        4.56136794, -5.45963017, -2.58172188,  3.01185164,  2.09271969,
        3.59110479, -2.66271615,  1.53287196,  0.3279889 ,  3.7957388 ,
       -0.42170341, -2.60743298,  0.82388388, -5.43361392, -0.88775517,
        1.59369119, -0.81030227, -0.56294716,  6.20710117,  0.20025215,
       -3.59746505,  0.259477  ,  1.39149207,  1.61993173, -3.11775503,
        1.11109864, -2.58381244, -2.24740834, -1.78756819,  1.60726667,
        9.31186998, -4.29413815,  0.53331956, -5.81558309,  0.17620202,
       -2.58166629,  0.93275271,  0.08439438,  0.96887137, -0.47639255,
       -0.51977737, -1.41891925, -0.65646523,  4.62694312,  0.59

In [10]:
n = training_data.shape[0]

# testing
test_train_res = np.ndarray((n, N))

for i in range(n):
    if i == 0:
        test_train_res[i] = np.dot(W_in, training_data[0])
    else:
        test_train_res[i] = next_training_node(
            r_prev=test_train_res[i-1],
            u_prev=training_data[i-1],
            hyperparams=hyperparams,
            W_r=W_r,
            W_in=W_in,
            delta_t=DEL_T
        )

test_modified_train_res = np.ndarray(test_train_res.shape)
for i in range(n):
    test_modified_train_res[i] = modify_node(test_train_res[i])

In [11]:
print(sum(sum(import_modified_train_res - test_modified_train_res)))
print(sum(sum(import_train_res - test_train_res)))

0.0
0.0
